***
$\mathbf{\text{MGSC 662 - GROUP PROJECT - Pulp Version}}$<br>
***
Team 10

### Objective function
find the mix of NBA players that maximizes team efficiency as per Player Efficiency Rating (PER) and other metrics while minimizing the cumulative roster cost using a dataset on NBA players data in 2017

+ Maximize the team efficiency as per Player Efficiency Rating (PER) and Expected Wins (EW)
> $Max. \sum \limits _{i=1} ^{394} \sum \limits _{j=1} ^{5}((PER)_{x_{i}} + EW_{x_{i}})*x_{i,j}$ <br>

Where $x_{i} \in {0,1}$ (player is chosen or not)

### Decision variables
$x_{ij}$ 

where i is: whether a play is drafter or not, where $x_{ij}\in {0,1}$ <br>
where j is:
+ PG: if the player's position is point guard where PG $\in {0,1}$
+ SG: if the player's position is shooting guard where SG $\in {0,1}$
+ SF: if the player's position is small forward where SF $\in {0,1}$
+ PF: if the player's position is power forward where PF $\in {0,1}$
+ C: if the player's position is center where C $\in {0,1}$


### Objective constraints

##### NBA salary constraint 
It is not a simple budget, rather it’s actually a “soft cap” which can be exceeded, but with a penalty cost (called a luxury tax line) a mechanism that helps control / deter over team spending. In 2017, the salary cap was 90.9 million dollars (for one team) and the tax level 1.5 dollar per dollar over. 

> $\sum \limits _{i=1} ^{394} (salary)_{i}*x_{i,j} \le 90.9$ 

> if salary > budget cap, the total budget = budget + luxury tax line 

##### Only undervalued players will be selected
> $x_i * salary_i \le b_0 + b_1*PER_i + b_2*age_i + b_3*WS_i + b_4*eFG_i$ 


#####  The total number of players on a team is 15
> $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,j} \le 15$

#####  The total number of player per position 
This is a soft constraint as the team manager can select the amount of player desired in each position
+ PG: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,PG} = 1$ <br>
+ SG: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,SG} = 2$ <br>
+ SF: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,SF} = 3$ <br>
+ PF: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,PF} = 4$ <br>
+ C: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,C} = 5$ <br>

#####  A player can only be assigned to the position that they are capable to play 
> $x_{i,j} \le PP_{i,j}$
    + i = 1,...,N 
    + j = 1,2,3,4,5

#####   Expected WS/48
In this iteration, total wins is predicted using Win Shares per 48 minutes and a prediction for the number of minutes that a player will play in the following season. <br/>
The formula to calculate a player's expected wins is:<br/>
> $EW_{i} = \frac{x_{i}*\frac{WS per 48_{i}}{48}*(Minutes Played_{i})^2*(1 - (1 - (\frac{1}{Minutes Played_{i}})^2)}{2}$ <br>


##### Players must be in the 10th percentile of minutes played
> $\frac{x_i}{MP_i} \le \frac{1}{quantile(MP, 0.1)}$ <br/>

#####   Starting players
In the National Basketball Association (NBA), two starting players are traditionally announced as guards, two as forwards, and one as a center.

So, we want to classify the starting players and make sure what we have them in each position

#####  The total number of starting players per position
> PG: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,PG}*SP_{i} = 1$ <br>
> SG: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,SG}*SP_{i} = 1$ <br>
> SF: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,SF}*SP_{i} = 1$ <br>
> PF: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,PF}*SP_{i} = 1$ <br>
> C: $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,C}*SP_{i} = 1$ <br>


##### Total Games Started
The team can have a maximum of 390 games started across all players (to ensure that there are not more starting-caliber players than the team can realistically afford).<br/>
> $\sum \limits _{i=1} ^{394}\sum \limits _{j=1} ^{5} x_{i,j}*GS_i \le 390$<br/><br/>


### limitations of Pulp:
Since this is not a linear constraint, the expected in shares have to be calculated before hand and added to the dataframe to be utilized in the objective function. Overall, we should be able to come down to the same solution but we can see that seeting this problem in Gurobi is more straightforward and streamlined than it is in Pulp.




In [1]:
import numpy as np
import pulp as plp
import pandas as pd 
from pulp import *
import gurobipy
from gurobipy import *

df = pd.read_csv (r"C:\Users\Sophie\Downloads\players.csv")
df['Salary'] = df['Salary']/1000000


Using license file C:\Users\Sophie\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-01
No parameters matching '_test' found


In [2]:
### this is just a way to see what the index of a specific column is in the dataset

col_name = 'WS'

index_no = df.columns.get_loc(col_name) 
  
print("Index of {} column in given dataframe is : {}".format(col_name, index_no))

Index of WS column in given dataframe is : 25


###### SETTING UP COLUMNS FOR NON LINEAR CONSTRAINTS WORKAROUND

1. Expected salary predictions
> $salary = -0.0394 + 0.2787*PER + 0.0269*age + 1.1325*WS + -0.0229*eFG$
2. Expected win shares
> $EW_{i} = \frac{x_{i}*(WS per 48_{i}/48)*(Minutes Played_{i})^2*(1 - (1 - (\frac{1}{Minutes Played_{i}})^2)}{2}$
3. Player position matrix 
4. Starting players

In [3]:
##### Expected Salary
E_S = []

players = range(len(df))
I = players

for i in I:
    PER = df.iloc[i, 11]
    age = df.iloc[i, 6]
    WS = df.iloc[i, 25]
    eFG = df.iloc[i, 40]
    
    Estimated_salary = -0.0394 + 0.2787*PER + 0.0269*age + 1.1325*WS + -0.0229*eFG
    E_S.append(Estimated_salary) #estimated salary column is added

df2 = df.assign(ES = E_S)
df2 #new dataframe


##### Expected win shares
E_W = []

for i in I:
    WS = df.iloc[i, 26] #WS/48 variable in dataset
    MP = df.iloc[i, 10] #MP variable in dataset
    
    expected_wins = (MP/2)*(WS/48)*MP*(1 - (1- 1/MP)*(1- 1/MP)) #EW formula from Gurobi
    E_W.append(expected_wins)

df3 = df2.assign(EW = E_W)


###### player position matrix

BB2 = df3[['ID', 'Player Name', 'Pos', 'Salary', 'PER', 'ES', 'WS', 'G', 'EW', 'GS', 'MP']]

players_POS = BB2.assign(PG = [1 if a == 'PG' else 0 for a in BB2['Pos']])
players_POS = players_POS.assign(SG = [1 if a == 'SG' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(SF = [1 if a == 'SF' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(PF = [1 if a == 'PF' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(C = [1 if a == 'C' else 0 for a in players_POS['Pos']])


#### Starting players
players_POS['GS_GP'] = players_POS.GS / players_POS.G
top_starting = players_POS['GS_GP'].quantile(0.8) ## this means players that start 100% of their games

players_POS = players_POS.assign(Start_P = [1 if a >= top_starting else 0 for a in players_POS['GS_GP']] )
 

###### SETTING UP THE PROBLEM

In [4]:
########################### DECISION VARIBLES ################################

#Establishing the variables 
positions = ['PG', 'SG', 'SF', 'PF', 'C']
players = range(len(players_POS))

I = players
J = range(len(positions))

X = plp.LpVariable.dicts(name = 'x', indexs=(I), cat=plp.LpBinary)

#Setting up the problem
prob1 = LpProblem(name = "basketball_positions")
prob1.sense = LpMaximize


############################# CONSTRAINTS ##################################

#1 - max 15 players
prob1 += lpSum(X[i] for i in I) <= 15

#2 - salary cap 
SALARY_CAP = 90.9
salary = players_POS['Salary'].tolist()
prob1 += lpSum(X[j]*salary[j] for j in I) <= SALARY_CAP, 'salary'


#3 - only select players if they are undervalued
estimated_salary = players_POS['ES'].tolist()
prob1 += lpSum(X[j]*salary[j] - X[j]*estimated_salary[j] for j in J) <= 0

#4 - position constraints
pointG = players_POS['PG'].tolist()
prob1 += lpSum(X[i]*pointG[i] for i in I) == 1

shootingG = players_POS['SG'].tolist()
prob1 += lpSum(X[i]*shootingG[i] for i in I) == 2

shootingF = players_POS['SF'].tolist()
prob1 += lpSum(X[i]*shootingF[i] for i in I) == 3

pointF = players_POS['PF'].tolist()
prob1 += lpSum(X[i]*pointF[i] for i in I) == 4

center = players_POS['C'].tolist()
prob1 += lpSum(X[i]*center[i] for i in I) == 5

#5 - players must have been in at least the 10th percentile for minutes played 
threshold = players_POS['MP'].quantile(0.1) #287 minutes played
minutes_played = players_POS['MP'].tolist()

prob1 += lpSum(X[i]*minutes_played[i] - threshold >= 0 for i in I) #they must have played more minutes then the threshold


#6 - Games started constraint
total_games_started = 390
games_started = players_POS['GS'].tolist()

prob1 += lpSum(X[i]*games_started[i] for i in I) <= 390


#7 - Starting players constraint
starting_player = players_POS['Start_P'].tolist()

prob1 += lpSum(X[i]*pointG[i]*starting_player[i] for i in I) == 1
prob1 += lpSum(X[i]*shootingG[i]*starting_player[i] for i in I) == 1
prob1 += lpSum(X[i]*shootingF[i]*starting_player[i] for i in I) == 1
prob1 += lpSum(X[i]*pointF[i]*starting_player[i] for i in I) == 1
prob1 += lpSum(X[i]*center[i]*starting_player[i] for i in I) == 1



############################### OBJECTIVE ###################################

#maximizing the PER and Win shares
PER = players_POS['PER'].tolist()
EW = players_POS['EW'].tolist()
prob1 += lpSum(X[i]*(PER[i]+EW[i]) for i in I)


C:\Users\Sophie\anaconda3\lib\site-packages\pulp\pulp.py:1521: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [5]:
prob1.solve(GUROBI(timeLimit=1))

Changed value of parameter TimeLimit to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14 rows, 394 columns and 1598 nonzeros
Model fingerprint: 0x43685cde
Variable types: 0 continuous, 394 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 8e+01]
  Objective range  [5e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 276.0526979
Presolve removed 2 rows and 168 columns
Presolve time: 0.00s
Presolved: 12 rows, 226 columns, 637 nonzeros
Found heuristic solution: objective 397.5099271
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 4.052909e+02, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

1

In [6]:
# The status of the solution is printed to the screen
print("=" * 19, "\nSolution Status:", plp.LpStatus[prob1.status])


# Results
print("objective value (Expected Wins plus PER): ", str(value(prob1.objective)))

print("-" * 19 + "\nOptimal Solution:")
for v in prob1.variables():
    if value(v) > 0:
        print(v.name, "=", v.varValue)


print("-" * 19 + "\nPayroll:")

#payroll and salary constraint
for c in list(prob1.constraints.values()):
    if c.name == 'salary':
        print('Salary constraint loose:', c.value())
        payroll = (SALARY_CAP - (-c.value()))*1000000
        print('Total payroll: $', payroll)

Solution Status: Optimal
objective value (Expected Wins plus PER):  402.1979270833341
-------------------
Optimal Solution:
x_137 = 1.0
x_162 = 1.0
x_166 = 1.0
x_179 = 1.0
x_180 = 1.0
x_193 = 1.0
x_206 = 1.0
x_231 = 1.0
x_235 = 1.0
x_282 = 1.0
x_32 = 1.0
x_328 = 1.0
x_332 = 1.0
x_84 = 1.0
x_99 = 1.0
-------------------
Payroll:
Salary constraint loose: -0.1368770000000119
Total payroll: $ 90763123.0


In [7]:
### PRINT OUT ONLY THE NAMES OF THE PLAYERS ON THE OPTIMAL TEAM

indicies = [137,162,166,179,180,196,206,231,235,282,32,328,332,84,99]

team = []
for i in range(len(indicies)):
    team.append(players_POS.loc[indicies[i], "Player Name"])
print("Optimal Team:\n")
for i in team:
    print(i)

Optimal Team:

Isaiah Thomas
JaVale McGee
Jeff Withey
Joel Bolomboy
Joel Embiid
Jrue Holiday
Kawhi Leonard
Larry Nance
Lou Williams
Nikola Jokic
Boban Marjanovic
Sam Dekker
Shabazz Muhammad
DeMar DeRozan
Dirk Nowitzki


In [8]:
## PRINT OUT DESCRIPTION OF PLAYERS

indicies = [137,162,166,179,193,205,206,235,32,332,346,43,55,78,84]
       
for i in range(len(indicies)):
    print("Player name:", players_POS.loc[indicies[i], "Player Name"],
         "\n PER:", players_POS.loc[indicies[i], "PER"],
         "\n Position:", players_POS.loc[indicies[i], "Pos"],
         "\n Salary (in millions): $", players_POS.loc[indicies[i], "Salary"],
         "\n Starting_player ?", players_POS.loc[indicies[i], 'Start_P'])
    print("-" * 19)


Player name: Isaiah Thomas 
 PER: 26.5 
 Position: PG 
 Salary (in millions): $ 6.261395 
 Starting_player ? 1
-------------------
Player name: JaVale McGee 
 PER: 25.2 
 Position: C 
 Salary (in millions): $ 2.116955 
 Starting_player ? 0
-------------------
Player name: Jeff Withey 
 PER: 18.8 
 Position: C 
 Salary (in millions): $ 1.57732 
 Starting_player ? 0
-------------------
Player name: Joel Bolomboy 
 PER: 19.7 
 Position: PF 
 Salary (in millions): $ 1.312611 
 Starting_player ? 0
-------------------
Player name: Josh Huestis 
 PER: 26.1 
 Position: PF 
 Salary (in millions): $ 1.471382 
 Starting_player ? 0
-------------------
Player name: Karl-Anthony Towns 
 PER: 26.0 
 Position: C 
 Salary (in millions): $ 6.21684 
 Starting_player ? 1
-------------------
Player name: Kawhi Leonard 
 PER: 27.5 
 Position: SF 
 Salary (in millions): $ 18.868625 
 Starting_player ? 1
-------------------
Player name: Lou Williams 
 PER: 21.4 
 Position: SG 
 Salary (in millions): $ 7.0 
 S

In [9]:
for i in indicies:
    print(players_POS['EW'][i])

12.574947916667837
3.461718749999983
1.8428645833332757
0.18703125000000015
0.1683854166666664
12.686031250001587
13.604249999999638
6.105093749999733
1.7062500000000371
2.7152708333331557
4.059854166666837
0.06830208333333342
0.4590312500000006
2.631625000000043
9.059104166667597
